# Задание 1. Шифр Хилла.

In [935]:
import numpy as np
from sympy import Matrix
from random import randint

In [936]:
local_alphabet = ['б', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у',
 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', ' ']
n = len(local_alphabet)
print(n)

31


### Ценное сообщение из 12 символов:

In [937]:
message = "я люблю хлеб"

### 3 матрицы-ключа:

In [938]:
key_matrix2 = np.array([[38, 21], [29, 16]])
key_matrix3 = np.array([[17, -39, -9], [-27, -90, 20], [-94, -9, -18]])
key_matrix4 = np.array([[-30, -45, -15, 76], [67, 30, 90, 73], [36, -66, 20, -5], [-97, 26, -20, -69]])

### Убедимся, что определители матриц не имеют общих делителей с n:

In [939]:
det_matrix2 = round(np.linalg.det(key_matrix2))
det_matrix3 = round(np.linalg.det(key_matrix3))
det_matrix4 = round(np.linalg.det(key_matrix4))

print(f"Определители для матриц: 2x2: {det_matrix2}, 3x3: {det_matrix3}, 4x4: {det_matrix4}")
print(f"Условие выполнено: {all(np.gcd(det_matrix_i, n) == 1 for det_matrix_i in (det_matrix2, det_matrix3, det_matrix4))}")

Определители для матриц: 2x2: -1, 3x3: 196827, 4x4: 59833990
Условие выполнено: True


### Зашифруем сообщение с помощью каждого из ключей:

In [940]:
def encrypt(key_matrix, msg):
        message_indices = [local_alphabet.index(char) for char in msg]
        key_size = key_matrix.shape[0]
        
        while len(message_indices) % key_size != 0:
            message_indices.append(local_alphabet.index(" "))

        encrypted_indices = []
        for i in range(0, len(message_indices), key_size):
            block = np.array(message_indices[i : i + key_size])
            encrypted_block = np.dot(key_matrix, block)
            for j in range(len(encrypted_block)):
                if encrypted_block[j] != 0:
                    encrypted_block[j] = encrypted_block[j] % len(local_alphabet)
            encrypted_indices.extend(encrypted_block)
            
        return "".join([local_alphabet[i] for i in encrypted_indices])


enc2, enc3, enc4 = (encrypt(key_matrix_i, message) for key_matrix_i in (key_matrix2, key_matrix3, key_matrix4))
print(f"Сообщения:\n1. {enc2}\n2. {enc3}\n3. {enc4}")

Сообщения:
1. эхбэёццчопрэ
2. уржщпэёънпнж
3. чъёляцешффиы


### Теперь расшифруем сообщения: 

In [941]:
def decrypt(key_matrix, enc):
    dim = key_matrix.shape[0]
    p = []
    dec = ''
    k_inv = np.array(Matrix(key_matrix).inv_mod(n))
    for i in range(len(enc)):
        if i % dim == 0 and i != 0:
            c = k_inv.dot(p) % n
            for j in c:
                dec += local_alphabet[j]
            p = []
        p.append(local_alphabet.index(enc[i]))
    c = k_inv.dot(p) % n
    for j in c:
        dec += local_alphabet[j]
        
    return dec

dec2, dec3, dec4 = (decrypt(key_matrix_i, enc_i) for key_matrix_i, enc_i in zip((key_matrix2, key_matrix3, key_matrix4), (enc2, enc3, enc4)))
print(f"Сообщения:\n1. {dec2}\n2. {dec3}\n3. {dec4}")

Сообщения:
1. я люблю хлеб
2. я люблю хлеб
3. я люблю хлеб


### Сымитируем вредоносное вмешательство в зашифрованные сообщения:

In [942]:
def hack(enc):
    indexes = []
    chars = []
    listed_mes = list(enc)

    while len(indexes) != 3:
        index = np.random.randint(0, len(enc))
        char = local_alphabet[np.random.randint(0, n)]
        if (index not in indexes) and (char != enc[index]):
            indexes.append(index)
            chars.append(char)

    for i in range(0, 3):
        listed_mes[indexes[i]] = chars[i]

    return "".join(listed_mes)


hacked_enc2, hacked_enc3, hacked_enc4 = (hack(enc_i) for enc_i in (enc2, enc3, enc4))
print(f"Сообщения:\n1. {hacked_enc2}\n2. {hacked_enc3}\n3. {hacked_enc4}")

Сообщения:
1. эхбпёцьчоярэ
2. урйщпзуънпнж
3. чъёлърешффиё


### Расшифруем получившиеся сообщения: 

In [943]:
hacked_dec2, hacked_dec3, hacked_dec4 = (decrypt(key_matrix_i, enc_i) for key_matrix_i, enc_i in zip((key_matrix2, key_matrix3, key_matrix4), (hacked_enc2, hacked_enc3, hacked_enc4)))

print(f"Сообщения:\n1. {hacked_dec2}\n2. {hacked_dec3}\n3. {hacked_dec4}")

Сообщения:
1. я ыеблыфрчеб
2. рппрнчтюблеб
3. я люиуолрчщь
